In [1]:
#from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
import time
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib.pyplot as plt
import numpy as np
import math
####################you will need to change some paths here!#####################
#list of input directories
dir_storm_info='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
dir_flux = 'F:/data/model_data/oaflux/data_v3/daily/turbulence/'
dir_cmc = 'F:/data/sst/cmc/CMC0.2deg/v2/'
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
##where to get the data through opendap, use these directories instead
#dir_cmc = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.1deg/v3/'
#dir_flux = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/WHOI_OAFlux/version3/daily/lh_oaflux/'
#the latest ccmp is from www.remss.com but they do not have an opendap server so you can use this instead:
#dir_ccmp='https://podaac-opendap.jpl.nasa.gov/opendap/allData/ccmp/L3.0/flk/'

#################################################################################
import geopy.distance
from math import sin, pi
from scipy import interpolate

#functions for running storm data
def interpolate_storm_path(dsx):
    #after calculating the distance from the storm it became clear that the storm data is every 6 hours, no matter 
    #how much it may have moved.  So if the storm moved 300 km in 6 hr, when calculating the distance to the storm
    #there were points on the storm track that showed large distances because of the separation to the 6hrly storm points
    #this subroutine interpolates the storm path onto a higher spatial resolution
    #the new storm dataset is carefully put into an identical format with i2 and j2 as dims to match the old format
    ynew = []
    tnew = []
    xnew = []
    dsx['lon'] = (dsx.lon-180) % 360 - 180 #put -180 to 180
    for istep in range(1,dsx.lon.shape[1]):
        dif_lat = dsx.lat[0,istep]-dsx.lat[0,istep-1]
        dif_lon = dsx.lon[0,istep]-dsx.lon[0,istep-1]
        x,y,t = dsx.lon[0,istep-1:istep+1].values,dsx.lat[0,istep-1:istep+1].values,dsx.time[0,istep-1:istep+1].values
        if abs(dif_lat)>abs(dif_lon):
            isign = np.sign(dif_lat)
            if abs(dif_lat)>0.75:
                ynew1 = np.arange(y[0], y[-1], isign.data*0.75)
                f = interpolate.interp1d(y, x, assume_sorted=False)
                xnew1 = f(ynew1)
                f = interpolate.interp1d(y, t, assume_sorted=False)
                tnew1 = f(ynew1)
            else:
                xnew1,ynew1,tnew1 = x,y,t
            xnew,ynew,tnew = np.append(xnew,xnew1),np.append(ynew,ynew1),np.append(tnew,tnew1) 
        else:
            isign = np.sign(dif_lon)
            if abs(dif_lon)>0.75:
                iwrap_interp = 1
                if (x[0]<-90) & (x[-1]>90):
                    iwrap_interp = -1
                    x[0]=x[0]+360
                if (x[0]>90) & (x[-1]<-90):
                    iwrap_interp = -1
                    x[-1]=x[-1]+360
                xnew1 = np.arange(x[0], x[-1], iwrap_interp*isign.data*0.75)
                f = interpolate.interp1d(x, y, assume_sorted=False)
                ynew1 = f(xnew1)
                f = interpolate.interp1d(x, t, assume_sorted=False)
                tnew1 = f(xnew1)
                xnew1 = (xnew1 - 180) % 360 - 180 #put -180 to 180
            else:
                xnew1,ynew1,tnew1 = x,y,t
            xnew,ynew,tnew = np.append(xnew,xnew1),np.append(ynew,ynew1),np.append(tnew,tnew1) 
    i2,j2=xnew.shape[0],1
    tem = np.expand_dims(xnew, axis=0)
    xx = xr.DataArray(tem.T,dims=['i2','j2'])
    tem = np.expand_dims(ynew, axis=0)
    yy = xr.DataArray(tem.T,dims=['i2','j2'])
    tem = np.expand_dims(tnew, axis=0)
    tt = xr.DataArray(tem.T,dims=['i2','j2'])
    dsx_new = xr.Dataset({'lon':xx.T,'lat':yy.T,'time':tt.T})
    return dsx_new

def get_dist_grid(lat_point,lon_point,lat_grid,lon_grid):
    #this routine takes a point and finds distance to all points in a grid of lat and lon
    #it is slowwwwwww
    dist_grid = np.empty(lat_grid.shape)    
    coords_1 = (lat_point, lon_point)  
    for i in range(0,lat_grid.shape[0]):
        for j in range(0,lat_grid.shape[1]):
            coords_2 = (lat_grid[i,j], lon_grid[i,j])  
            arclen_temp = geopy.distance.geodesic(coords_1, coords_2).km  #distance in km       
            dist_grid[i,j]=arclen_temp
    return dist_grid


def closest_dist(ds_in,ds_storm): 
# m.garcia-reyes 2.4.2019, edited c.gentemann 2.4.2019
# calculate distance closest storm point
# point given as tla,tlo.... storm is in the program
# initialize distances (in km)
    ds_storm['lon'] = (ds_storm.lon + 180) % 360 - 180
    dsx_input = ds_storm.copy(deep=True)
    ds_storm_new = interpolate_storm_path(dsx_input)       
    tdim,xdim,ydim=ds_storm_new.lat.shape[1], ds_in.analysed_sst[0,:,0].shape[0], ds_in.analysed_sst[0,0,:].shape[0]
    dx_save=np.zeros([tdim,xdim,ydim])
    dx_grid,dy_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)
    lon_grid,lat_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)
    min_dist_save = np.zeros([xdim,ydim])*np.nan
    min_index_save = np.zeros([xdim,ydim])*np.nan
    min_time_save = np.zeros([xdim,ydim])*np.nan
    #for each location of the storm calculate the difference for all values in box
    for ipt in range(0,ds_storm_new.lat.shape[1]):  # all storm values
        dist_tem_grid = get_dist_grid(ds_storm_new.lat[0,ipt].values,ds_storm_new.lon[0,ipt].values,lat_grid,lon_grid)
        dx_save[ipt,:,:]=dist_tem_grid       
    #now go through each value in box and find minimum storm location/day
    for j in range(0,ds_in.lon.shape[0]):
        for i in range(0,ds_in.lat.shape[0]):
            imin = np.argmin(dx_save[:,i,j])
            min_dist_save[i,j]=dx_save[imin,i,j]
            min_index_save[i,j]=imin
            min_time_save[i,j]=ds_storm_new.time[0,imin]
    return min_dist_save,min_index_save,min_time_save

def calculate_storm_mask(ds_sst,lats,lons):
#make a mask for the storm and only keep data within -4 and 10 degrees of storm track
#this was written before I had calculated the closest_dist which is probably a better mask to use
    iwrap_mask = 0
    if (ds_sst.lon.max().values>170) & (ds_sst.lon.min().values<-170):
        iwrap_mask=1
    print(ds_sst.lon.min().values,ds_sst.lon.max().values)
#okay, now ds_storm is array with right lat/lon for storm so create mask now
    ds_mask = ds_sst.copy(deep=True)
    ds_mask['storm_mask']=ds_mask.analysed_sst*0
    ds_mask = ds_mask.fillna(0)
    ds_mask['storm_mask'] = ds_mask['storm_mask'].astype(int,copy=True)
    for i in range(0,lats.shape[0]):
        if lats[i]>0:   #northern hemi on right, southers on left
            lons1,lons2=lons[i]-4,lons[i]+10
        else:
            lons1,lons2=lons[i]-10,lons[i]+4
        lats1,lats2=lats[i]-10,lats[i]+10
        if i==0:
            print('lons1,lons2:',iwrap_mask,lons1.data,lons2.data)
        if lons1<-180:
            ds_mask['storm_mask'].loc[dict(lon=(ds_mask.lon < lons2) | (ds_mask.lon > lons1+360), lat=slice(lats1,lats2))] = -1
        elif lons2>180:
            ds_mask['storm_mask'].loc[dict(lon=(ds_mask.lon < lons2-360) | (ds_mask.lon > lons1), lat=slice(lats1,lats2))] = -1
        else:
            if iwrap_mask==1:
                ds_mask.coords['lon'] = np.mod(ds_mask['lon'], 360)
                ds_mask = ds_mask.sortby(ds_mask.lon)
                ds_mask['storm_mask'].loc[dict(lon=slice(lons1+360,lons2+360), lat=slice(lats1,lats2))] = -1
                ds_mask.coords['lon'] = (ds_mask.coords['lon'] + 180) % 360 - 180
            else:
                ds_mask['storm_mask'].loc[dict(lon=slice(lons1,lons2), lat=slice(lats1,lats2))] = -1
    return ds_mask


In [2]:
#dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
#date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
#dx=0.25
#dy=0.25
#dx_offset = -179.875
#dy_offset = -78.3750


In [ ]:
isave_mld_year = 0 #init MLD monthly data read flag
for root, dirs, files in os.walk(dir_storm_info, topdown=False):
    if root[len(dir_storm_info):len(dir_storm_info)+1]=='.':
        continue
    for name in files:
        if not name.endswith('.nc'):
            continue
        filename=os.path.join(root, name)
        print(filename[36:39],filename[31:35])
        inum_storm=int(filename[36:39])
        iyr_storm=int(filename[31:35])
#        if iyr_storm!=2007: # or iyr_storm<2003:
#            continue
        print(name,filename)
        ds_storm_info = xr.open_dataset(filename)
        lats = ds_storm_info.lat[0,:]
        lons = ds_storm_info.lon[0,:]  #lons goes from 0 to 360
        lons = (lons + 180) % 360 - 180 #put -180 to 180
        dysince = ds_storm_info.time
        ds_storm_info.close()
        
#make lat and lon of storm onto 25 km grid for below
        lons = (((lons - .125)/.25+1).astype(int)-1)*.25+.125
        lats = (((lats + 89.875)/.25+1).astype(int)-1)*.25-89.875
        
        iwrap=0
#calculate size of box to get data in
        minlon,maxlon = min(lons.values)-10, max(lons.values)+10
        minlat,maxlat = min(lats.values)-10, max(lats.values)+10

        ydim_storm = round((maxlat - minlat)/.25).astype(int)
        new_lat_storm = np.linspace(minlat, maxlat, ydim_storm)
        if (minlon<-90 and maxlon>=90) or (minlon<-180 and maxlon<0):  #this storm wraps  keep everythig 0 to 360 then wrap data at very end
            iwrap = 1
            lons2 = np.mod(lons, 360)
            minlon, maxlon = min(lons2.values)-10, max(lons2.values)+10
            xdim_storm = round((maxlon - minlon)/.25).astype(int)
            new_lon_storm = np.linspace(minlon, maxlon, xdim_storm)
        else:
            xdim_storm = round((maxlon - minlon)/.25).astype(int)
            new_lon_storm = np.linspace(minlon, maxlon, xdim_storm)

        print(iwrap,minlon,maxlon)
        print(iwrap,xdim_storm, new_lon_storm[:5],new_lon_storm[-5:])

        
        dims=lats.shape
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        min_date = min(tem_date)+dt.timedelta(days=-5)
        max_date = max(tem_date)+dt.timedelta(days=5)
        minjdy = min_date.timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min_date.year #create new time array that can be queried for year etc
        minmon =min_date.month #create new time array that can be queried for year etc
        minday =min_date.day #create new time array that can be queried for year etc
        maxjdy = max_date.timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max_date.year  #create new time array that can be queried for year etc
        print(minyear,minjdy,maxyear,maxjdy)
        
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)+30             #calculate ssts for 30 days after storm

        #print(tdim,xdim,ydim)            
        
        #print('sst_out_sv',sst_out_sv.shape)
        for i in range(0,tdim):
            storm_date = dt.datetime(minyear,minmon,minday)+dt.timedelta(days=i)+dt.timedelta(hours=12)
            #print(storm_date)
            
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)

#sst data   
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
            filename = dir_cmc + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            ds_day=xr.open_dataset(filename,drop_variables=['analysis_error','sea_ice_fraction'])
            if iwrap==1:  #data is -180 to 180 for sst, so need to bring to 0 to 360 when wrapped
                ds_day.coords['lon'] = np.mod(ds_day['lon'], 360)
                ds_day = ds_day.sortby(ds_day.lon)
            ds_day.close()
            ds_day = ds_day.where(ds_day['mask'] == 1.) 
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            #ds_storm['time']=storm_date
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            if i==0:
                ds_storm_sst = ds_storm
            else:
                ds_storm_sst = xr.concat([ds_storm_sst,ds_storm],dim='time')

#sst climatology  --- this isn't used, should remove from dataset in next round
#            if storm_date.timetuple().tm_yday==366:
#                sjdy = '365'
#            filename='F:/data/sst/cmc/CMC0.2deg/v2/climatology/clim1993_2016' + sjdy.zfill(3) + '-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
#            ds_day=xr.open_dataset(filename,drop_variables=['analysis_error','sea_ice_fraction','sq_sst'])
#            ds_day = ds_day.rename({'analysed_sst':'analysed_sst_clim','mask':'mask_clim'}) #, inplace = True)            
#            if iwrap==1:  #data is -180 to 180 for sst, so need to bring to 0 to 360 when wrapped
#                ds_day.coords['lon'] = np.mod(ds_day['lon'], 360)
#                ds_day = ds_day.sortby(ds_day.lon)
#            ds_day.close()
#            ds_day = ds_day.where(ds_day['mask_clim'] == 1.) 
#            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
#            ds_storm = ds_storm.assign_coords(time=storm_date)
#            if iwrap==1:
#                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
#            if i==0:
#                ds_storm_sst_clim = ds_storm
#            else:
#                ds_storm_sst_clim = xr.concat([ds_storm_sst_clim,ds_storm],dim='time')           
            
#ccmp wind data, no masked data, a complete field
            lyr, idyjl = 2015,1
#            storm_date = dt.datetime(2015,1,1)
            syr, smon, sdym, sjdy=str(storm_date.year),str(storm_date.month),str(storm_date.day),str(storm_date.timetuple().tm_yday)
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon.zfill(2) + fname_tem      
            ds=xr.open_dataset(ccmp_filename,drop_variables=['nobs'])
            ds_day = ds.mean(dim='time')     #take average across all 6 hourly data fields
            ds_day = ds_day.rename({'longitude':'lon','latitude':'lat'}) #, inplace = True)            
            if iwrap==0:
                ds_day.coords['lon'] = (ds_day.coords['lon'] + 180) % 360 - 180
                ds_day = ds_day.sortby(ds_day.lon)
            ds.close()
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm = ds_storm.assign_coords(time=storm_date)
            if i==0:
                ds_storm_ccmp = ds_storm
            else:
                ds_storm_ccmp = xr.concat([ds_storm_ccmp,ds_storm],dim='time')
              
#ocean mixed layer depth from monthly data GODAS NOAA, lon 0 to 360, monthly data so interp to day
            #dir_godas='https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/godas/'
            dir_godas = 'f:/data/model_data/godas/'
            if isave_mld_year != storm_date.year:
                filename = dir_godas + 'dbss_obml.' + syr + '.nc'
                ds_day_mld=xr.open_dataset(filename)
                if iwrap==0:
                    ds_day_mld.coords['lon'] = (ds_day_mld.coords['lon'] + 180) % 360 - 180
                    ds_day_mld = ds_day_mld.sortby(ds_day_mld.lon)
                ds_day_mld.close()
                isave_mld_year = storm_date.year
            ds_storm = ds_day_mld.interp(time = storm_date, lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm = ds_storm.assign_coords(time=storm_date)
            if i==0:
                ds_storm_mld = ds_storm
            else:
                ds_storm_mld = xr.concat([ds_storm_mld,ds_storm],dim='time')            
            
#latent heat flux data, masked already set to NaN                
            filename = dir_flux + 'lh_oaflux_' + syr + '.nc';
            ds=xr.open_dataset(filename,drop_variables=['err'])
            ds_day = ds.sel(time = storm_date.timetuple().tm_yday)  #select day of year from annual file
            if iwrap==0:
                ds_day.coords['lon'] = (ds_day.coords['lon'] + 180) % 360 - 180
                ds_day = ds_day.sortby(ds_day.lon)
            ds.close()
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm['time']=storm_date
            if i==0:
                ds_storm_lhf = ds_storm
            else:
                ds_storm_lhf = xr.concat([ds_storm_lhf,ds_storm],dim='time')

#sensible heat flux data , masked already set to NaN                
            filename = dir_flux + 'sh_oaflux_' + syr + '.nc';
            ds=xr.open_dataset(filename,drop_variables=['err'])
            ds_day = ds.sel(time = storm_date.timetuple().tm_yday)  #select day of year from annual file
            if iwrap==0:
                ds_day.coords['lon'] = (ds_day.coords['lon'] + 180) % 360 - 180
                ds_day = ds_day.sortby(ds_day.lon)
            ds.close()
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm['time']=storm_date
            if i==0:
                ds_storm_shf = ds_storm
            else:
                ds_storm_shf = xr.concat([ds_storm_shf,ds_storm],dim='time')

#surface humid flux data   , masked already set to NaN              
            filename = dir_flux + 'qa_oaflux_' + syr + '.nc';
            ds=xr.open_dataset(filename,drop_variables=['err'])
            ds_day = ds.sel(time = storm_date.timetuple().tm_yday)  #select day of year from annual file
            if iwrap==0:
                ds_day.coords['lon'] = (ds_day.coords['lon'] + 180) % 360 - 180
                ds_day = ds_day.sortby(ds_day.lon)
            ds.close()
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm['time']=storm_date
            if i==0:
                ds_storm_qa = ds_storm
            else:
                ds_storm_qa = xr.concat([ds_storm_qa,ds_storm],dim='time')

#air temp flux data   , masked already set to NaN              
            filename = dir_flux + 'ta_oaflux_' + syr + '.nc';
            ds=xr.open_dataset(filename,drop_variables=['err'])
            ds_day = ds.sel(time = storm_date.timetuple().tm_yday)  #select day of year from annual file
            if iwrap==0:
                ds_day.coords['lon'] = (ds_day.coords['lon'] + 180) % 360 - 180
                ds_day = ds_day.sortby(ds_day.lon)
            ds.close()
            ds_storm = ds_day.interp(lat = new_lat_storm,lon = new_lon_storm)
            if iwrap==1:
                ds_storm.coords['lon'] = (ds_storm.coords['lon'] + 180) % 360 - 180
            ds_storm['time']=storm_date
            if i==0:
                ds_storm_ta = ds_storm
            else:
                ds_storm_ta = xr.concat([ds_storm_ta,ds_storm],dim='time')
                
#        ds_all = xr.merge([ds_storm_ccmp, ds_storm_mld, ds_storm_lhf, ds_storm_shf, ds_storm_ta, ds_storm_qa, ds_storm_sst, ds_storm_sst_clim])
        ds_all = xr.merge([ds_storm_ccmp, ds_storm_mld, ds_storm_lhf, ds_storm_shf, ds_storm_ta, ds_storm_qa, ds_storm_sst])

        #calculate mask
        print('caluculating mask')
        ds_mask = calculate_storm_mask(ds_all,lats,lons)
        ds_all['storm_mask']=ds_mask['storm_mask']
        #dist to storm
        print('calculating dist')
        dist,index,stime = closest_dist(ds_all,ds_storm_info)
        dtem=xr.DataArray(dist, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
        ds_all['dist_from_storm_km']=dtem
        dtem=xr.DataArray(index, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
        ds_all['closest_storm_index']=dtem
        dtem=xr.DataArray(stime, coords={'lat': ds_mask.lat.values, 'lon':ds_mask.lon.values}, dims=('lat', 'lon'))
        ds_all['closest_storm_time']=dtem

        
        filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_combined_data.nc'
        ds_all.to_netcdf(filename)
        print('out:',filename)
     # filename = dir_out + str(iyr_storm) + '/' + 'str(inum_storm)' + '_other_data.nc'
    


In [ ]:
ds_all.closest_storm_time.plot()

In [ ]:
#test distance from storm
#read in some data from a storm that wraps
iwrap=0
iyr_storm,inum_storm=2002,1
filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_combined_data.nc'
ds = xr.open_dataset(filename)
ds.close()
ds_sst = ds.copy(deep=True)   #make a deep copy
dsx = xr.open_dataset('F:/data/tc_wakes/database/info/'+str(iyr_storm)+'/'+ str(inum_storm).zfill(3) +'annual_storm_info.nc')
dsx.close()
#dist,index,stime = closest_dist(ds_sst,dsx)




In [ ]:
#plt.pcolormesh(dx_save[imin,:,:])
print(imin)
plt.plot(dx_save[:,i,j])

In [ ]:
#this interpolater better than numpy interpolater because you don't have to force 
iyr_storm, inum_storm = 2002,2
dsx = xr.open_dataset('F:/data/tc_wakes/database/info/'+str(iyr_storm)+'/'+ str(inum_storm).zfill(3) +'annual_storm_info.nc')
dsx['lon'] = (dsx.lon + 180) % 360 - 180 #put -180 to 180
dsx.close()
dif_lat = dsx.lat[0,-1]-dsx.lat[0,0]
dif_lon = dsx.lon[0,-1]-dsx.lon[0,0]
print(dif_lat.values,dif_lon.values)
if abs(dif_lat)>abs(dif_lon):
    print('norm')
    x,y,t = dsx.lat[0,:].values,dsx.lon[0,:].values,dsx.time[0,:].values
    isign = np.sign(dif_lat)
    xnew = np.arange(x[0], x[-1], isign.data*0.25)
    f = interpolate.interp1d(x, y, assume_sorted=False)
    ynew = f(xnew)
    f = interpolate.interp1d(x, t, assume_sorted=False)
    tnew = f(xnew)
else:
    print('test')
    x,y,t = dsx.lat[0,:].values,dsx.lon[0,:].values,dsx.time[0,:].values
    isign = np.sign(dif_lon)
    ynew = np.arange(y[0], y[-1], isign*0.25)
    f = interpolate.interp1d(y, x, assume_sorted=False)
    xnew = f(ynew)
    f = interpolate.interp1d(t, t, assume_sorted=False)
    tnew = f(tnew)
    
print(x[0:5])
print(y[0:5])
print(xnew[0:5])
print(ynew[0:5])
print(t[0:5])
print(tnew[0:5])
plt.plot(x,y,'.-')
plt.plot(xnew,ynew,'r.')


In [ ]:
#this interpolater better than numpy interpolater because you don't have to force 
#test do step by step through storm
            

iyr_storm, inum_storm = 2002,13
dsx = xr.open_dataset('F:/data/tc_wakes/database/info/'+str(iyr_storm)+'/'+ str(inum_storm).zfill(3) +'annual_storm_info.nc')
dsx.close()
dsx['lon'] = (dsx.lon-180) % 360 - 180 #put -180 to 180
dsx_input = dsx.copy(deep=True)
dsx_new = interpolate_storm_path(dsx_input)    
print(dsx.lon[0,:])
plt.plot(dsx.lon[0,:],dsx.lat[0,:],'b.-')
plt.plot(dsx_new.lon[0,:],dsx_new.lat[0,:],'r.')
#plt.plot(xnew,ynew,'r.')


In [ ]:
print(dsx)
print(dsx_new)
#dsx_new['lon']=xnew
#plt.plot(dsx.lon[0,:],dsx.time[0,:],'b-.')
#plt.plot(xnew,tnew,'r.')
#print(dsx.lon[0,0:10])
#print(xnew[10:25])
#print(ynew[10:25])
#print(tnew[10:25])

In [ ]:
iwrap=0
iyr_storm,inum_storm=2002,5
filename = dir_out + str(iyr_storm) + '/' + str(inum_storm).zfill(3) + '_combined_data.nc'
ds = xr.open_dataset(filename)
ds.close()
ds_sst = ds.copy(deep=True)   #make a deep copy
dsx = xr.open_dataset('F:/data/tc_wakes/database/info/'+str(iyr_storm)+'/'+ str(inum_storm).zfill(3) +'annual_storm_info.nc')
dsx.close()

ds_storm = dsx
ds_in = ds_sst
ds_storm['lon'] = (ds_storm.lon + 180) % 360 - 180
ds_storm_new = interpolate_storm_path(ds_storm)       
tdim,xdim,ydim=ds_storm_new.lat.shape[1], ds_in.analysed_sst[0,:,0].shape[0], ds_in.analysed_sst[0,0,:].shape[0]
dx_save=np.zeros([tdim,xdim,ydim])
print(ds_in.analysed_sst.shape)
dx_grid,dy_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)
lon_grid,lat_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)
min_dist_save = np.zeros(ds_in.analysed_sst[0,:,:].shape)*np.nan
min_index_save = np.zeros(ds_in.analysed_sst[0,:,:].shape)*np.nan
#for each location of the storm calculate the difference for all values in box
for ipt in range(0,ds_storm_new.lat.shape[1]):  # all storm values
    dist_tem_grid = get_dist_grid(ds_storm_new.lat[0,ipt].values,ds_storm_new.lon[0,ipt].values,lat_grid,lon_grid)
    dx_save[ipt,:,:]=dist_tem_grid
#now go through each value in box and find minimum storm location/day
for j in range(0,ds_in.lon.shape[0]):
    for i in range(0,ds_in.lat.shape[0]):
        imin = np.argmin(dx_save[:,i,j])
        min_dist_save[i,j]=dx_save[imin,i,j]
        min_index_save[i,j]=imin


In [ ]:
#plt.pcolormesh(dx_save[20,:,:],vmin = 0, vmax = 1000)
#plt.pcolormesh(min_dist_save,vmin = 0, vmax = 1000)
plt.pcolormesh(dist,vmin = 0, vmax = 3000)

In [ ]:
ds_in = ds_sst
ds_storm = dsx
ds_storm['lon'] = (ds_storm.lon + 180) % 360 - 180
tdim,xdim,ydim=ds_storm.lat.shape[1], ds_in.analysed_sst[0,:,0].shape[0], ds_in.analysed_sst[0,0,:].shape[0]
dx_save=np.zeros([tdim,xdim,ydim])
print(ds_in.analysed_sst.shape)
lon_grid,lat_grid = np.meshgrid(ds_in.lon.values,ds_in.lat.values)

ipt = 1
lat_point,lon_point = ds_storm.lat[0,ipt],ds_storm.lon[0,ipt]
dist_grid = np.empty(lat_grid.shape)    
coords_1 = (lat_point, lon_point)  
for i in range(0,lat_grid.shape[0]):
    for j in range(0,lat_grid.shape[1]):
        coords_2 = (lat_grid[i,j], lon_grid[i,j])  
        arclen_temp = geopy.distance.geodesic(coords_1, coords_2).km  #distance in km       
        dist_grid[i,j]=arclen_temp


In [ ]:
plt.pcolormesh(dist_grid)

In [ ]:
ds_storm_ccmp.vwnd[20,:,:].plot()

In [ ]:
filename2 = 'f:/data/model_data/godas/dbss_obml.2002.nc'
ds_day_mld=xr.open_dataset(filename2)
ds_day_mld.close()
ds_day_mld

In [ ]:
isave_mld_year

In [ ]:
print(dir_godas + 'dbss_obml.' + syr + '.nc')
ds_day=xr.open_dataset(dir_godas + 'dbss_obml.' + syr + '.nc')

In [ ]:
#OLD CODE using netcdf and calculating location directly rather than using xarray
###### dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
for root, dirs, files in os.walk(dir_storm_info, topdown=False):
#    for ii in range(12,13): 
    for name in files:
    #    name = files[ii]
#    for name in files:
        fname_in=os.path.join(root, name)
        fname_out=dir_out + fname_in[31:39] + '_all_25km.nc'
        inum_storm=int(fname_in[36:39])
        iyr_storm=int(fname_in[31:35])
        if iyr_storm>2003 or iyr_storm<2003:
            continue
#        if iyr_storm==2011 and inum_storm<15:
#            continue
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]  #lons goes from 0 to 360
        dysince = dsx.time
        #minlon=min(lons[0,:].values)-10
        #maxlon=max(lons[0,:].values)+10
        #minlat=min(lats[0,:].values)-10
        #maxlat=max(lats[0,:].values)+10
        
        iwrap=0
        minlon=min(lons.values)-10
        maxlon=max(lons.values)+10
        minlat=min(lats.values)-10
        maxlat=max(lats.values)+10
        if minlon<10 and maxlon>350:  #wrapping around meridion need to cal new min/max lon
            minlon=max(lons[lons<180].values)+10
            maxlon=min(lons[lons>180].values)-10
            iwrap=1 #set flag for wraparound
        
        #here is a fix for when a storm goes from 350 across 360 to 1 2 longitude
#        iwrap=0
#        print('first and last!',lons[0,1].values,lons[0,-1].values)
#        if abs(min(lons[0,:].values)-max(lons[0,:].values))>180:
#            lons1=lons[0,:].values-10>180
#            lons2=lons[0,:].values+10<180
#            maxlon=min(lons[0,lons1].values-10)
#            minlon=max(lons[0,lons2].values+10)
#            print('wrapped',minlon,maxlon)
#            iwrap=1
            #wrap_lons = ((lons+180) % 360) - 180        
            #maxlon=max(wrap_lons[0,:].values)+10 #this will find the positive maximum
            #minlon=min(wrap_lons[0,:].values)-10
            #if minlon<0:
            #    maxlon=min(wrap_lons[0,:].values)-10+360
            #    minlon=max(wrap_lons[0,:].values)+10

        print('min/max lon lat',minlon,maxlon,minlat,maxlat)

        ix1=int(round((minlon-dx_offset)/dx))
        ix2=int(round((maxlon-dx_offset)/dx))
        iy1=int(round((minlat-dy_offset)/dy))
        iy2=int(round((maxlat-dy_offset)/dy))
        if iy2 > 628:
            iy2=628
        if iy1 < 1:
            iy1=1    
        if ix1 < 0:
            ix1 = ix1 + 1440
        if ix2 < 0:
            ix2 = ix2 + 1440
        print(minlon,maxlon,minlat,maxlat)
        xdim=ix2-ix1
        if iwrap==1:  #wraps around so make sure xdim reflects that
            xdim=ix1-ix2+1440
        ydim=iy2-iy1
        
        dims=lats.shape
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        minjdy = min(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min(tem_date).year #create new time array that can be queried for year etc
        maxjdy = max(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max(tem_date).year  #create new time array that can be queried for year etc
        print(minyear,minjdy,maxyear,maxjdy)
        
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)+30


        print(tdim,ix1,ix2,iy1,iy2,xdim,ydim)      
               
        sst_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        sst_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndu_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndv_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndu_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndv_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        
        print('sst_out_sv',sst_out_sv.shape)
        for i in range(0,tdim):
            storm_date = tem_date[0]+dt.timedelta(days=i)
            #print(storm_date)
            
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)
            
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon.zfill(2) + fname_tem      

            #flux data
            lh_flux_filename = dir_flux + 'lh_oaflux_' + syr + '.nc';
            sh_flux_filename = dir_flux + 'sh_oaflux_' + syr + '.nc';
            ta_flux_filename = dir_flux + 'ta_oaflux_' + syr + '.nc';
            qa_flux_filename = dir_flux + 'qa_oaflux_' + syr + '.nc';
  #          fname='F:\data\model_data\oaflux\data_v3\daily\radiation_1985-2009\sw_isccp_2004.nc';
 #           [nswrs]=ncread(fname,'nswrs',[1 1 idy],[360 180 1]);
            
            if storm_date.timetuple().tm_yday==366:
                sjdy = '365'
            clim_filename='F:/data/sst/jpl_mur/v4.1/clim/clim2_' + sjdy.zfill(3) +'_2003_2013_MUR-GLOB-v02.0-fv04.1.nc'
            ccmp_clim_filename='F:/data/sat_data/ccmp/v02.0/clim/ccmp_daily_clim_' + sjdy.zfill(3) +'.nc'
#            print(ccmp_filename)
#            print(ccmp_clim_filename)

#ccmp wind            
            nc_fid = Dataset(ccmp_filename, 'r')
            nc_fid2 = Dataset(ccmp_clim_filename, 'r') 
            tem = nc_fid.variables['uwnd'][:,iy1:iy2,:]  #read in data all longitude, limited latitude
            tem = np.mean(tem,axis=0)                     #take average across all 6 hourly data fields
            wndu = np.append(tem[:,ydim:],tem[:,:ydim], axis=1) #switch from 0-360 to -180 to 180  ydim is half of xdim
            tem = nc_fid.variables['vwnd'][:,iy1:iy2,:]
            tem = np.mean(tem,axis=0)
            wndv = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            mlat_ccmp = nc_fid.variables['latitude'][iy1:iy2]
            tem = nc_fid.variables['longitude'][:]
            mlon_ccmp = np.append(tem[ydim:],tem[:ydim], axis=0)  
            mlon_save = mlon_ccmp[:]
            mlon_ccmp = ((mlon_ccmp - 180) % 360) - 180  #make -180 to 180 rather than 0 360
            tem = nc_fid2.variables['av_u'][iy1:iy2,:]
            wndu_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            tem = nc_fid2.variables['av_v'][iy1:iy2,:]
            wndv_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            nc_fid.close()
            nc_fid2.close()           


            #flux data
            ds = xr.open_dataset(lh_flux_filename)
            ds_subset = ds.sel(time = idyjl)
            ds_res = ds_subset.interp(latitude = new_lat,longitude = mlon_save)
           # nc_fid4 = Dataset(sh_flux_filename, 'r')
           # nc_fid5 = Dataset(ta_flux_filename, 'r')
           # nc_fid6 = Dataset(qa_flux_filename, 'r')

            #[lhf1]=ncread(fname,'lhtfl',[1 1 idy],[360 180 1]);
            #[Tair1]=ncread(fname,'tmp2m',[1 1 idy],[360 180 1]);
            #[Qair1]=ncread(fname,'hum2m',[1 1 idy],[360 180 1]);  
            #[shf1]=ncread(fname,'shtfl',[1 1 idy],[360 180 1]);

            
            
            
#            if i==0:
#                print('i=0',iy1,iy2,ix1,ix2,iy2-iy1,ix2-ix1)
            if ix1<=1440 and ix2<=1440 and iwrap==0:
#                if i==0:
#                    print('inside1',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix1:ix2]
                wndu_clim_out = wndu_clim[:,ix1:ix2]
                wndv_out = wndv[:,ix1:ix2]
                wndv_clim_out = wndv_clim[:,ix1:ix2]
            if ix1>1440 and ix2>1440 and iwrap==0:
 #               if i==0:
#                    print('inside2',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix1-1440:ix2-1440]
                wndu_clim_out = wndu_clim[:,ix1-1440:ix2-1440]
                wndv_out = wndv[:,ix1-1440:ix2-1440]
                wndv_clim_out = wndv_clim[:,ix1-1440:ix2-1440]
            if ix1<=1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside3',iy1,iy2,ix1,ix2)

                tem1 = wndu[:,ix1:]
                tem2 = wndu[:,:ix2-1440]
                wndu_out = np.append(tem1,tem2, axis=1)
                tem1 = wndv[:,ix1:]
                tem2 = wndv[:,:ix2-1440]
                wndv_out = np.append(tem1,tem2, axis=1)
                tem1 = wndu_clim[:,ix1:]
                tem2 = wndu_clim[:,:ix2-1440]
                wndu_clim_out = np.append(tem1,tem2, axis=1)
                tem1 = wndv_clim[:,ix1:]
                tem2 = wndv_clim[:,:ix2-1440]
                wndv_clim_out = np.append(tem1,tem2, axis=1)

            if ix1<=1440 and ix2>1440 and iwrap==1:
#                if i==0:
#                    print('inside1',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix2-1440:ix1]
                wndu_clim_out = wndu_clim[:,ix2-1440:ix1]
                wndv_out = wndv[:,ix2-1440:ix1]
                wndv_clim_out = wndv_clim[:,ix2-1440:ix1]

            wndu_out_sv[i,:,:]=wndu_out
            wndv_out_sv[i,:,:]=wndv_out
            wndu_clim_out_sv[i,:,:]=wndu_clim_out
            wndv_clim_out_sv[i,:,:]=wndv_clim_out
            
#sst data   

            nc_fid = Dataset(mur_filename, 'r')
            mlat = nc_fid.variables['lat'][1149:16849]
            ilat_mur1 = np.argmin(abs(mlat-mlat_ccmp.min()))-12
            #print('mlat first point:', mlat[ilat_mur1_tem])
            ilat_mur2 = np.argmin(abs(mlat-mlat_ccmp.max()))+13
            mlat = mlat[ilat_mur1:ilat_mur2]
            sst = nc_fid.variables['analysed_sst'][0,ilat_mur1:ilat_mur2,:]
            mlon = nc_fid.variables['lon'][:]
            nc_fid.close()

            nc_fid2 = Dataset(clim_filename, 'r')                      
            sst_clim = nc_fid2.variables['sst'][ilat_mur1:ilat_mur2,:]
            nc_fid2.close()
                       
            coarseness = 25
            temp = mlon.reshape((mlon.shape[0] // coarseness, coarseness))
            coarse_mlon = np.mean(temp, axis=(1), dtype=np.float64)
            temp = mlat.reshape((mlat.shape[0] // coarseness, coarseness))
            coarse_mlat = np.mean(temp, axis=(1), dtype=np.float64)
            temp = sst.reshape((sst.shape[0] // coarseness, coarseness, sst.shape[1] // coarseness, coarseness))
            coarse_sst = np.mean(temp, axis=(1,3), dtype=np.float64)
            temp = sst_clim.reshape((sst_clim.shape[0] // coarseness, coarseness, sst_clim.shape[1] // coarseness, coarseness))
            coarse_sst_clim = np.mean(temp, axis=(1,3), dtype=np.float64)

            #need to recalculate iy1 and iy2 because of offset made earlier to read less of file
#            iy1=np.argmin(abs(coarse_mlat-minlat))
#            iy2=np.argmin(abs(coarse_mlat-maxlat))
#            ydim=iy2-iy1   
#            print(coarse_mlat[0],coarse_mlat[-1])
#            print(iy1,iy2,ydim,minlat,maxlat)

#            if i==0:
#                print('i=0',ix1,ix2,iy2-iy1,ix2-ix1)
            if ix1<=1440 and ix2<=1440 and iwrap==0:
#                if i==0:
#                    print('inside1',ix1,ix2)
                sst_out = coarse_sst[:,ix1:ix2]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix1:ix2]
                sst_clim_out = coarse_sst_clim[:,ix1:ix2]
            if ix1>1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside2',ix1,ix2)
                sst_out = coarse_sst[:,ix1-1440:ix2-1440]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix1-1440:ix2-1440]
                sst_clim_out = coarse_sst_clim[:,ix1-1440:ix2-1440]
            if ix1<=1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside3',ix1,ix2)
                tem1 = coarse_sst[:,ix1:]
                tem2 = coarse_sst[:,:ix2-1440]
                sst_out = np.append(tem1,tem2, axis=1)
                mlat_out = coarse_mlat[:]
                mlon1 = coarse_mlon[ix1:]
                mlon2 = coarse_mlon[:ix2-1440]
                print(mlon1.shape,mlon2.shape)
                mlon_out = np.append(mlon1,mlon2, axis=0)               
                tem1 = coarse_sst_clim[:,ix1:]
                tem2 = coarse_sst_clim[:,:ix2-1440]
                sst_clim_out = np.append(tem1,tem2, axis=1)

            if ix1<=1440 and ix2>1440 and iwrap==1:
 #               if i==0:
 #                   print('inside1',ix1,ix2)
                sst_out = coarse_sst[:,ix2-1440:ix1]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix2-1440:ix1]
                sst_clim_out = coarse_sst_clim[:,ix2-1440:ix1]

            if i==0:
                print('sst',sst_out.shape,'coarse',coarse_sst.shape,wndu_out.shape)
                print('sst',sst_out.shape,'sst_sv',sst_out_sv.shape)
                print('mlon',mlon_out.shape,'mlat',mlat_out.shape)
            #sst_sv[i,:,:]=sst-sst_clim
            sst_out_sv[i,:,:]=sst_out
            sst_clim_out_sv[i,:,:]=sst_clim_out


  
 
            
            
        ilen=len(fname_in)
        
        dif_dys=[0]*tdim
        for i in range(0,tdim):
            dif_dys[i] = i

        print('file out:',fname_out)
        #f.close()
        f = Dataset(fname_out,'w', format='NETCDF4') 
        tempgrp = f.createGroup('data')
        tempgrp.setncattr_string('start time',str(tem_date[0]))
        tempgrp.createDimension('t', tdim)
        tempgrp.createDimension('y', ydim)
        tempgrp.createDimension('x', xdim)

    #tem_date[i]
        sst_netcdf = tempgrp.createVariable('sst', 'f4', ('t', 'y', 'x'))
        sst_clim_netcdf = tempgrp.createVariable('sst_clim', 'f4', ('t', 'y', 'x'))
        wndu_netcdf = tempgrp.createVariable('wndu', 'f4', ('t', 'y', 'x'))
        wndv_netcdf = tempgrp.createVariable('wndv', 'f4', ('t', 'y', 'x'))
        wndu_clim_netcdf = tempgrp.createVariable('wndu_clim', 'f4', ('t', 'y', 'x'))
        wndv_clim_netcdf = tempgrp.createVariable('wndv_clim', 'f4', ('t', 'y', 'x'))
        longitude = tempgrp.createVariable('lon', 'f4', 'x')
        latitude = tempgrp.createVariable('lat', 'f4', 'y')  
        time = tempgrp.createVariable('time', 'i4', 't')
        
        sst_netcdf[:] = sst_out_sv
        sst_clim_netcdf[:] = sst_clim_out_sv
        wndu_netcdf[:] = wndu_out_sv
        wndv_netcdf[:] = wndv_out_sv
        wndu_clim_netcdf[:] = wndu_clim_out_sv
        wndv_clim_netcdf[:] = wndv_clim_out_sv
        latitude[:] = mlat_out
        longitude[:] = mlon_out
        time[:]=dif_dys
        f.close()
    
